prefix_ratio = len(prefix) / min(pred, gt) 

is_prefix = is_prefix(pred, gt) | is_prefix(gt, pred)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
from ipywidgets import interact, widgets
from xlwings.utils import chunk
import numpy as np

from plotter import read_results, get_group_columns, filter_target_columns, plot_dropdown, plot_dropdown_with_group_by, make_interaction

%matplotlib inline

In [2]:
from pathlib import Path
base_path = Path("/mnt/data/kolomyttseva/long-contex-eval/output/rag_results/python_kotlin/chunk_score")

# full_file_path = base_path / "full_file"
# fixed_line_path = base_path / "fixed_line"
# langchain_path = base_path / "langchain"

paths_list = [
    "results_embed_full_file.jsonl",
    "results_fixed_line.jsonl",
    "results_langchain.jsonl",
    # "results_scorers_splitters.jsonl"
]

In [3]:
def merge_df(base_path, paths_list: list[str]):
    dataframes = []
    for path in paths_list:
        full_path = base_path / path
        if not Path(full_path).exists():
            raise FileNotFoundError(f"Path {full_path} does not exist")
        df = pd.read_json(full_path, orient="records", lines=True)
        dataframes.append(df)
    final_dataframe = pd.concat(dataframes, ignore_index=True)
    return final_dataframe

In [4]:
df = merge_df(base_path, paths_list)

# Drop all columns for which we don't want aggregation
group_columns = get_group_columns(df)

results = read_results(df, group_columns)

In [5]:
# from pathlib import Path
# base_path = Path("/mnt/data/kolomyttseva/long-contex-eval/output/rag_results/chunk_score")

# full_file_path = base_path / "full_file"
# fixed_line_path = base_path / "fixed_line"
# langchain_path = base_path / "langchain"

'''
fixed_line:
    - chunk_completion_file.jsonl
        - scorer: bm25
        - splitter: word_splitter
        - chunk_lines_size = [8, 16, 32, 64, 128]
        - chunk_completion_file = [True, False]
        - completion_last_chunk_size = 32
    - completion_last_chunk_size.jsonl
        - scorer: bm25
        - splitter: word_splitter
        - chunk_lines_size = [8, 16, 32, 64, 128]
        - chunk_completion_file = True
        - completion_last_chunk_size = [8, 16, 32, 64, 128]

full_file:
    - bm25_word_splitter.jsonl
        - scorer: bm25
        - splitter: word_splitter
    - dense_scorer_word_splitter.jsonl
        - scorer: dense
        - splitter: word_splitter
    - scorers_splitters.jsonl
        - scorer: iou/bm25
        - splitter: line_splitter/word_splitter/model_tokenizer

langchain:
    - langchain.jsonl
        - scorer: bm25
        - splitter: word_splitter
        - chunk_lines_size = [8, 16, 32, 64, 128]
        - chunk_completion_file = True
        - completion_last_chunk_size = 32
'''

'\nfixed_line:\n    - chunk_completion_file.jsonl\n        - scorer: bm25\n        - splitter: word_splitter\n        - chunk_lines_size = [8, 16, 32, 64, 128]\n        - chunk_completion_file = [True, False]\n        - completion_last_chunk_size = 32\n    - completion_last_chunk_size.jsonl\n        - scorer: bm25\n        - splitter: word_splitter\n        - chunk_lines_size = [8, 16, 32, 64, 128]\n        - chunk_completion_file = True\n        - completion_last_chunk_size = [8, 16, 32, 64, 128]\n\nfull_file:\n    - bm25_word_splitter.jsonl\n        - scorer: bm25\n        - splitter: word_splitter\n    - dense_scorer_word_splitter.jsonl\n        - scorer: dense\n        - splitter: word_splitter\n    - scorers_splitters.jsonl\n        - scorer: iou/bm25\n        - splitter: line_splitter/word_splitter/model_tokenizer\n\nlangchain:\n    - langchain.jsonl\n        - scorer: bm25\n        - splitter: word_splitter\n        - chunk_lines_size = [8, 16, 32, 64, 128]\n        - chunk_comp

### Analyze scorers and splitters

chunk_score:
- chunker: full_file
- scorer: iou/bm25/dense
- splitter: line_splitter/word_splitter/model_tokenizer

In [6]:
full_file_results = results[results["chunker"] == "full_file"]

#### Analyse scorers for each splitter

In [ ]:
plot_params = {
    'plot_by': 'scorer',
}
make_interaction(results=full_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='scorer', description='plot_by'), Dropdown(description='metric', options=('em…

In [ ]:
plot_params = {
    'plot_by': 'scorer',
    'group_by': 'splitter'

}
make_interaction(results=full_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown_with_group_by, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='scorer', description='plot_by'), Text(value='splitter', description='group_b…

#### Analyse splitters for each scorer

In [10]:
plot_params = {
    'plot_by': 'splitter',
}
make_interaction(results=full_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='splitter', description='plot_by'), Dropdown(description='metric', options=('…

In [11]:
plot_params = {
    'plot_by': 'splitter',
    'group_by': 'scorer'
}
make_interaction(results=full_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown_with_group_by, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='splitter', description='plot_by'), Text(value='scorer', description='group_b…

### Analyse chunk_completion_file

chunk_score:
- chunker: fixed_line
- scorer: bm25
- splitter: word_splitter
- chunk_lines_size = [8, 16, 32, 64, 128]
- chunk_completion_file = [True, False]
- completion_last_chunk_size = 32

In [28]:
chunk_completion_file_results = results[results["chunker"] == "fixed_line"]

In [ ]:
plot_params = {
    'plot_by': 'chunk_lines_size',
}
make_interaction(results=chunk_completion_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 )

interactive(children=(Text(value='chunk_lines_size', description='plot_by'), Dropdown(description='metric', op…

In [ ]:
plot_params = {
    'plot_by': 'chunk_lines_size',
    'group_by': 'chunk_completion_file',

}
make_interaction(results=chunk_completion_file_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown_with_group_by, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 )

interactive(children=(Text(value='chunk_lines_size', description='plot_by'), Text(value='chunk_completion_file…

### Analyse chunkers

chunk_score:
- chunker: full_file/fixed_line/langchain
- scorer: bm25
- splitter: word_splitter

setup only for fixed_line/langchain:
- chunk_lines_size = [8, 16, 32, 64, 128]
- chunk_completion_file = True
- completion_last_chunk_size = 32

In [33]:
bm25_word_splitter_results = results[(results["scorer"] == "bm25") 
                                     & (results["splitter"] == "word_splitter") 
                                     & (results["chunk_completion_file"] == True)]

fixed_line_results = bm25_word_splitter_results[(bm25_word_splitter_results["chunker"] == "full_file") | (bm25_word_splitter_results["chunker"] == "fixed_line" )]
langchain_results = bm25_word_splitter_results[(bm25_word_splitter_results["chunker"] == "full_file") | (bm25_word_splitter_results["chunker"] == "langchain")]

#### Full_file VS Fixed_Line

In [ ]:
plot_params = {
    'plot_by': 'chunk_lines_size',
}
make_interaction(results=fixed_line_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='chunk_lines_size', description='plot_by'), Dropdown(description='metric', op…

#### Full_file VS Langchain

In [ ]:
plot_params = {
    'plot_by': 'chunk_lines_size',
}
make_interaction(results=langchain_results, 
                 group_columns=group_columns, 
                 dropdown=plot_dropdown, 
                 plot_params=plot_params,
                 metrics=['em', 'is_prefix', 'prefix_ratio'],
                 delete_columns=["embed_model"],
                 )

interactive(children=(Text(value='chunk_lines_size', description='plot_by'), Dropdown(description='metric', op…